In [89]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import env
import os

In [90]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    '''
    This function takes in user credentials from an env.py file and a database name and creates a connection to the Codeup database through a connection string 
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'


curriculum_sql_query =  '''
                    SELECT * FROM logs
                    LEFT JOIN cohorts ON logs.cohort_id = cohorts.id;
                    '''

def query_curriculum_data():
    '''
    This function uses the get_connection function to connect to the SQL database and returns the data read into a pandas dataframe
    '''
    return pd.read_sql(curriculum_sql_query,get_connection('curriculum_logs'))

def get_curriculum_data():
    '''
    This function checks for a local file and reads it into a pandas dataframe, if it exists. If not, it uses the get_connection & query functions to query the data and write it locally to a csv file
    '''
    # If csv file exists locally, read in data from csv file.
    if os.path.isfile('curriculum_logs.csv'):
        df = pd.read_csv('curriculum_logs.csv', index_col=0)
        
    else:
        
        # Query and read data from database
        df = query_curriculum_data()
        
        # Cache data
        df.to_csv('curriculum_logs.csv')
        
    return df

In [91]:
# use a function to connect to and pull in data from SQL
df = get_curriculum_data()
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0


In [92]:
# check info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900223 entries, 0 to 900222
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   path        900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   id          847330 non-null  float64
 7   name        847330 non-null  object 
 8   slack       847330 non-null  object 
 9   start_date  847330 non-null  object 
 10  end_date    847330 non-null  object 
 11  created_at  847330 non-null  object 
 12  updated_at  847330 non-null  object 
 13  deleted_at  0 non-null       float64
 14  program_id  847330 non-null  float64
dtypes: float64(4), int64(1), object(10)
memory usage: 109.9+ MB


In [93]:
# drop some of the unnecessary columns
df = df.drop(columns=['id', 'slack', 'deleted_at'])

In [94]:
# check number of unique user ids where cohort_id is null
df[df.cohort_id.isnull()].user_id.nunique()

78

#### There are some columns with null values. My guess is that this is instructors as there are only 78 unique ids that fall within this category and it makes sense that they would not be associated to any specific cohort

# Exercise 1

Label students by the program they are in.

In [95]:
df.name[df.program_id == 2].value_counts()

Staff         84031
Ceres         40730
Zion          38096
Jupiter       37109
Fortuna       36902
Voyageurs     35636
Ganymede      33844
Apex          33568
Deimos        32888
Teddy         30926
Hyperion      29855
Betelgeuse    29356
Ulysses       28534
Europa        28033
Xanadu        27749
Wrangell      25586
Andromeda     25359
Kalypso       23691
Yosemite      20743
Bash          17713
Luna          16623
Marco         16397
Sequoia        7444
Neptune        7276
Pinnacles      2158
Oberon         1672
Niagara         755
Mammoth         691
Name: name, dtype: int64

In [96]:
# create column that labels as data science if program_id is 3, otherwise web development
df['program'] = ['data_science' if x ==3 else 'web_development' for x in df.program_id]
df.head()

,date,time,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id,program
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,web_development
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,web_development
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,web_development
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,web_development
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,web_development


# Exercise 2

Is it possible to identify the user_id of a staff member?

#### I discovered there is a "name" for staff members so my theory about the columns with nulls being associated with staff seems to be incorrect

In [97]:
# check number of unique user ids where name is Staff
df[df.name == 'Staff'].user_id.nunique()

51

In [98]:
# create a staff subset
staff_subset = df[df.name == 'Staff']
staff_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84031 entries, 166284 to 900222
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        84031 non-null  object 
 1   time        84031 non-null  object 
 2   path        84031 non-null  object 
 3   user_id     84031 non-null  int64  
 4   cohort_id   84031 non-null  float64
 5   ip          84031 non-null  object 
 6   name        84031 non-null  object 
 7   start_date  84031 non-null  object 
 8   end_date    84031 non-null  object 
 9   created_at  84031 non-null  object 
 10  updated_at  84031 non-null  object 
 11  program_id  84031 non-null  float64
 12  program     84031 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 9.0+ MB


# Exercise 3

Identify students who are accessing our curriculum pages beyond the end of their time at Codeup.

In [106]:
# get user ids for students that are not staff and have a cohort id
df[(df.date > df.end_date) & (df.name != 'Staff') & (df.cohort_id.notnull())].user_id.unique()

array([  1,  11,  21,  26,  37,  40,  50,  51,  53,  60,  64,  66,  67,
        68,  69,  70,  71,  72,  75,  76,  77,  80,  81,  82,  83,  84,
        85,  87,  90,  91,  92,  93,  94,  95,  96,  97,  98, 101, 102,
        16,  42,  41,  17,  22,  15, 104,  47,  57,  10, 105, 106,  28,
       107,  65,  52,  33,  56, 108,  27,  38,  36, 109, 110, 112, 113,
       114, 115, 139, 140, 141, 143, 144, 146, 147, 148, 149, 150, 151,
       152, 153, 156, 161, 162, 163, 164, 165, 167, 168, 169, 170, 172,
       173, 174, 175, 177, 178, 179,  29,  19, 180,   6,  99,  43,  31,
         7, 181, 138,  35, 142,   5, 155, 154,  88, 209, 210,  18,   3,
        20, 118, 211, 212, 214, 216, 217, 131, 135, 128, 127, 124, 130,
       159,  12, 145, 241, 242, 136, 157, 171, 123,  25, 134, 119, 243,
       244, 245, 246, 160, 248, 252, 254, 255, 120, 257, 279, 280, 281,
        14, 253, 188, 186, 283, 184, 215, 278, 183, 204, 193, 195, 197,
       249, 206, 284, 121, 201, 285, 286, 311, 190, 312, 116, 20

# Exercise 4

Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2.

In [109]:
df.head()

,date,time,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id,program
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,web_development
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,web_development
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,web_development
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1.0,web_development
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2.0,web_development


In [111]:
# change date to datetime, set as index, and create a df of pages accessed
df.date = pd.to_datetime(df.date)
df = df.set_index(df.date)
pages = df.path.resample('d').count()
pages.head()

date
2018-01-26    572
2018-01-27    230
2018-01-28    170
2018-01-29    830
2018-01-30    652
Freq: D, Name: path, dtype: int64

In [112]:
# compute midband
midband = pages.ewm(span=30).mean()
# compute exponential stdev
stdev = pages.ewm(span=30).std()
# compute upper and lower bands
ub = midband + stdev*3
lb = midband - stdev*3

In [ ]:
# concatenate computations and pages together into one df
my_df = pd.concat([pages, midband, bb], axis=1)

In [82]:
# check the unique user_ids from staff subset
np.sort(staff_subset.user_id.unique())

array([  1,  11,  37,  40,  41,  53,  64, 146, 148, 211, 248, 257, 308,
       312, 314, 315, 370, 397, 404, 428, 430, 461, 480, 502, 513, 514,
       521, 539, 545, 546, 572, 581, 592, 618, 620, 630, 652, 653, 738,
       742, 744, 745, 813, 816, 854, 855, 893, 894, 951, 953, 980])

In [71]:
null_subset = df[df.cohort_id.isnull()]
null_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52893 entries, 411 to 899902
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        52893 non-null  object 
 1   time        52893 non-null  object 
 2   path        52893 non-null  object 
 3   user_id     52893 non-null  int64  
 4   cohort_id   0 non-null      float64
 5   ip          52893 non-null  object 
 6   name        0 non-null      object 
 7   start_date  0 non-null      object 
 8   end_date    0 non-null      object 
 9   created_at  0 non-null      object 
 10  updated_at  0 non-null      object 
 11  program_id  0 non-null      float64
dtypes: float64(2), int64(1), object(9)
memory usage: 5.2+ MB


In [83]:
# check the unique user_ids from null subset
np.sort(null_subset.user_id.unique())

array([ 48,  54,  58,  59,  61,  62,  63,  64,  73,  74,  78,  79,  86,
        88,  89, 100, 103, 111, 137, 166, 176, 213, 247, 317, 346, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 372, 375, 403, 406, 429, 544,
       644, 663, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723,
       724, 725, 726, 727, 728, 729, 731, 736, 744, 782, 810, 814, 815])

In [86]:
df[df.user_id == 368]

,date,time,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id
193832,2019-02-07,16:29:09,2.00.02_Navigating_Excel,368,NaN,97.105.19.58,NaN,NaN,NaN,NaN,NaN,NaN
194385,2019-02-08,16:34:44,2.00.01_Intro_Excel,368,NaN,97.105.19.58,NaN,NaN,NaN,NaN,NaN,NaN
194386,2019-02-08,16:34:57,2.00.02_Navigating_Excel,368,NaN,97.105.19.58,NaN,NaN,NaN,NaN,NaN,NaN
194387,2019-02-08,16:34:58,2.00.03_Import_Excel,368,NaN,97.105.19.58,NaN,NaN,NaN,NaN,NaN,NaN
194388,2019-02-08,16:35:01,2.00.02_Navigating_Excel,368,NaN,97.105.19.58,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
511239,2020-04-14,11:02:37,3-sql/11-more-exercises,368,NaN,173.173.113.51,NaN,NaN,NaN,NaN,NaN,NaN
511403,2020-04-14,12:51:32,3-sql/11-more-exercises,368,NaN,173.173.113.51,NaN,NaN,NaN,NaN,NaN,NaN
511500,2020-04-14,14:41:50,3-sql/11-more-exercises,368,NaN,173.173.113.51,NaN,NaN,NaN,NaN,NaN,NaN
512459,2020-04-15,11:49:39,3-sql/11-more-exercises,368,NaN,173.173.113.51,NaN,NaN,NaN,NaN,NaN,NaN
